In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os,sys
sys.path.insert(1, os.path.join(sys.path[0], '..', 'module'))

In [3]:
import wiki

path_base = '/Users/harangju/Developer/data/wiki/dumps/'
name_xml = 'enwiki-20190801-pages-articles-multistream.xml.bz2'
name_index = 'enwiki-20190801-pages-articles-multistream-index.txt.bz2'
path_xml = path_base + name_xml
path_index = path_base + name_index
dump = wiki.Dump(path_xml, path_index)

In [4]:
topic = 'biochemistry'
%time dump.load_page('Index of {} articles'.format(topic))
links = dump.links
dump.links[:5]

Dump: Loading index...
Dump: Loaded.
CPU times: user 1min 7s, sys: 2.16 s, total: 1min 9s
Wall time: 1min 9s


['Chemistry', 'Organism', 'Structure', 'Protein', 'Carbohydrate']

#### Test network generation

In [5]:
net = wiki.Net()
net.build_graph(dump, nodes=[str(l) for l in links])

wiki.Net: traversing Wikipedia...
wiki.Net: depth = 0
wiki.Net: len(queue) = 4390
wiki.Net: depth = 1
wiki.Net: len(queue) = 438wiki.Net: removing isolates...
wiki.Net: adding years...
wiki.Net: filling empty years...


In [6]:
[len(links), len(net.graph.nodes)]

[1221, 902]

In [7]:
net.save_graph('test.gexf')

In [8]:
new_net = wiki.Net()
new_net.load_graph('test.gexf')
len(new_net.graph.nodes)

902

In [9]:
[n for n in net.graph.nodes][:5]

['Chemical element', 'Chemistry', 'Chemical compound', 'Atom', 'Molecule']

In [10]:
[y for y in net.years][:5]

[-2500, -2000, -1999, -700, -500]

In [11]:
net._numbered = None
[n for n in net.numbered.nodes][:5]

[0, 1, 30, 173, 175]

#### Test clique to barcodes

In [12]:
net._cliques = None
%time net.cliques[:4]
%time net.cliques[:4]

CPU times: user 66.3 ms, sys: 1.03 ms, total: 67.3 ms
Wall time: 66.6 ms
CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.01 µs


[[0], [1], [30], [173]]

In [13]:
net.nodes_for_year[-2500]

[60, 206]

In [15]:
%time net.filtration

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 6.91 µs


Filtration with 8192 simplices

In [16]:
%time net.persistence

CPU times: user 5.16 ms, sys: 21 µs, total: 5.18 ms
Wall time: 5.19 ms


Reduced matrix with 8192 columns

In [17]:
net._barcodes = None
%time net.barcodes
%time net.barcodes

wiki.Net: computing barcodes... (skip negatives)
wiki.Net: barcode 8188/8192
CPU times: user 2.03 s, sys: 568 ms, total: 2.6 s
Wall time: 2.24 s
CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 11.2 µs


,dim,birth,death,birth simplex,death simplex,birth nodes,death nodes
0,0,-2500,inf,[Carbon],[],[Carbon],[]
1,0,-2500,800.0,[Nutrition],"[Ethanol, Ammonia]",[Nutrition],[Ethanol]
2,0,-2000,-2000.0,[Alcohol],"[Carbon, Alcohol]",[Alcohol],[Alcohol]
3,0,-2000,1669.0,[Pesticide],"[Phosphorus, Pesticide]",[Pesticide],[Phosphorus]
4,0,-2000,1600.0,[Medicine],"[Cell biology, Microscope]",[Medicine],[Cell biology]
5,0,-1999,-1999.0,[Pharmacist],"[Medicine, Pharmacist]",[Pharmacist],[Pharmacist]
6,0,-700,800.0,[Sugar],"[Sugar, Ethanol]",[Sugar],[Ethanol]
7,0,-500,1600.0,[Phylogenetics],"[Evolution, Phylogenetics]",[Phylogenetics],[Evolution]
8,0,-500,1599.0,[Microbiology],"[Medicine, Virology]",[Microbiology],[Virology]
9,0,-500,-360.0,[Sulfur],"[Chemical element, Sulfur]",[Sulfur],[Chemical element]


#### Test weighting with tf-idf model

In [18]:
import pickle
import gensim.utils as gu

path_save = '/Users/harangju/Developer/data/wiki/models/'
tfidf = gu.SaveLoad.load(path_save + 'tfidf.model')
dct = pickle.load(open(path_save + 'dict.model','rb'))

In [19]:
net.graph['Chemistry']

AtlasView({'Activation energy': {'weight': 1}, 'Alcohol': {'weight': 1}, 'Alkali': {'weight': 1}, 'Atomic mass unit': {'weight': 1}, 'Biochemistry': {'weight': 1}, 'Chemical biology': {'weight': 1}, 'Chemical property': {'weight': 1}, 'Cheminformatics': {'weight': 1}, 'Chemist': {'weight': 1}, 'Colloid': {'weight': 1}, 'Competitive inhibition': {'weight': 1}, 'Computational biology': {'weight': 1}, 'Computational chemistry': {'weight': 1}, 'Concentration': {'weight': 1}, 'Diffusion': {'weight': 1}, 'Dissociation constant': {'weight': 1}, 'Electron shell': {'weight': 1}, 'Electrophile': {'weight': 1}, 'Energy level': {'weight': 1}, 'Ester': {'weight': 1}, 'Ethanol': {'weight': 1}, 'Fatty acid': {'weight': 1}, 'Hydrophobe': {'weight': 1}, 'Isomer': {'weight': 1}, 'Mixture': {'weight': 1}, 'Molecular engineering': {'weight': 1}, 'Molecular orbital': {'weight': 1}, 'Neurotransmitter': {'weight': 1}, 'Nucleon': {'weight': 1}, 'Organic chemistry': {'weight': 1}, 'Organic compound': {'weight'

In [20]:
w_net = wiki.Net()
w_net.build_graph(dump, nodes=[str(l) for l in links], model=tfidf, dct=dct)

wiki.Net: traversing Wikipedia...
wiki.Net: depth = 0
wiki.Net: len(queue) = 4390
wiki.Net: depth = 1
wiki.Net: len(queue) = 438wiki.Net: removing isolates...
wiki.Net: adding years...
wiki.Net: filling empty years...
wiki.Net: calculating weights...


In [21]:
w_net.graph['Chemistry']

AtlasView({'Activation energy': {'weight': array([[0.35918677]])}, 'Alcohol': {'weight': array([[0.2417242]])}, 'Alkali': {'weight': array([[0.22986483]])}, 'Atomic mass unit': {'weight': array([[0.37801507]])}, 'Biochemistry': {'weight': array([[0.45564846]])}, 'Chemical biology': {'weight': array([[0.46953734]])}, 'Chemical property': {'weight': array([[0.4983038]])}, 'Cheminformatics': {'weight': array([[0.37195845]])}, 'Chemist': {'weight': array([[0.65778113]])}, 'Colloid': {'weight': array([[0.2864258]])}, 'Competitive inhibition': {'weight': array([[0.20522618]])}, 'Computational biology': {'weight': array([[0.20443557]])}, 'Computational chemistry': {'weight': array([[0.5475628]])}, 'Concentration': {'weight': array([[0.22337339]])}, 'Diffusion': {'weight': array([[0.27894364]])}, 'Dissociation constant': {'weight': array([[0.26312483]])}, 'Electron shell': {'weight': array([[0.31375614]])}, 'Electrophile': {'weight': array([[0.40621243]])}, 'Energy level': {'weight': array([[0